In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys


from typing import Dict, List
from tqdm import tqdm_notebook
from fake_useragent import UserAgent

In [2]:
url_main = "https://www.flightconnections.com/"
url_codes = "flights-from-boston-bos"

In [3]:
def get_soup(url):
    res = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    html = res.content
    soup = BeautifulSoup(html, 'lxml')
    return soup

In [4]:
soup = get_soup(url_main + url_codes)

In [5]:
popular_destinations_list_from_boston = soup.findAll("a", attrs={"class":"popular-destination btn"})
popular_destinations_list_from_boston_hidden = soup.findAll("a", attrs={"class":"popular-destination btn hide"})
popular_destinations_list_from_boston = popular_destinations_list_from_boston_hidden + popular_destinations_list_from_boston
# popular_destinations_list_from_boston

In [6]:
def get_airport_info(airport)->Dict[str,str]:
    name = airport.find("div", attrs={"class":"popular-destination-airport-name"}).text
    destination_code = re.findall("\((.*?)\)", name)[0]
    country = airport.find("img").get("title")
    
    destination_flights_per_month = airport.find("span", attrs={'class':"float-right"}).text.strip()
    destination_flights_per_month = re.findall('\d+', destination_flights_per_month)[0]
    if destination_flights_per_month.isdigit():
        destination_flights_per_month = int(destination_flights_per_month)
    
    return {"airport_name":name, "code": destination_code, "country":country, "flights_per_month": destination_flights_per_month}

In [7]:
airport_info = []

for airport in tqdm_notebook(popular_destinations_list_from_boston):
    airport = get_airport_info(airport)
    if airport["country"] == "United States":
        airport_info.append(airport)
    

<ipython-input-7-88fb6df9ee85>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for airport in tqdm_notebook(popular_destinations_list_from_boston):


In [8]:
result = pd.DataFrame(airport_info)
result.to_csv("../data/routes.csv")

FileNotFoundError: [Errno 2] No such file or directory: './data/routes.csv'